In [ ]:
# !pip install python-dotenv

In [15]:
from dotenv import load_dotenv
from langchain import HuggingFaceHub, LLMChain

load_dotenv()

True

In [8]:
hub = HuggingFaceHub(repo_id="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO")

c:\Users\alanj\Documents\GitHub\dynolearn\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [9]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.runnables import RunnablePassthrough

In [13]:
from langsmith import Client

client = Client()

In [22]:
tasks_template = """Only provide the list

{about}. Make a list of 5 suitable tasks to {teaching_task}.

Just list out each item 1 by 1 as a JSON list."""

tasks_prompt = PromptTemplate(
    template = tasks_template,
    input_variables=["about", "teaching_task"]
)

dialogue_template = """About: {about}

Question: What dialogue would be helpful in a game where the tasks are:

{tasks}

Answer: Let's think step by step."""

dialogue_prompt = PromptTemplate(
    template = dialogue_template, 
    input_variables=["about", "tasks"]
)

In [23]:
tasks_chain = tasks_prompt | hub | hub.bind(stop="\n6")

In [24]:
for chunk in tasks_chain.stream(    {
        "about": "James likes Paw Patrol", 
        "teaching_task": "teach james how to get ready for school"
    }):
    print(chunk, end="", flush=True)

 ]

In [16]:
tasks_chain = tasks_prompt | hub  | hub.bind(stop="\n6")
dialogue_chain = dialogue_prompt | hub 

chain = (
    {"tasks": tasks_chain, "about": PromptTemplate.from_template("James likes Paw Patrol")} | RunnablePassthrough.assign(dialogue=dialogue_chain)
)

chain.stream(
    {
        "about": "James likes Paw Patrol", 
        "teaching_task": "teach james how to get ready for school"
    }
)

{'tasks': ' the bathroom immediately and do his thing.',
 'about': StringPromptValue(text='James likes Paw Patrol'),
 'dialogue': ' Firstly, let\'s divide the task into smaller parts. The task is "the bathroom immediately and do his thing." This implies James has to move quickly to the bathroom and perform his tasks in time. He might be late for some reason, or he is in a hurry. Based on this, we can construct a dialogue.\n\nJames (nervously): C\'mon, I need to get to the bathroom immediately!\nPlayer: Okay, let\'s hurry up.'}